<h1>Here Starts the College cleaning :_:</h1>

In [504]:
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm import tqdm

In [505]:
def syn_gen(col_name):
    lst = [None]*25
    if col_name == 'nan':
        return ['nan']
    col_name = str(col_name)
    col_name = col_name.lower()
    name = col_name.split(',')[0]
    names = []
    names.append(name)                                           # As given
    
    name = re.subn('of|\sand|&', '', name, flags = re.IGNORECASE)[0]
    
    name = re.subn('\s+', ' ', name)[0]                          # Removing 'of' and 'and'
    names.append(name)
    names.append(''.join([word[0] for word in name.split()]).upper())
    
    if "'" in col_name:    
        names.append(''.join([word[0] for word in filter(None,re.split('\W+', name))]).upper())
    if col_name.count('.') > 1:
        names.append(''.join(re.split(r'\.\s?', col_name)[:-1]).upper())
        
        try:
            pattern = re.compile('\s?univ\S*\s?|\s?coll\S*\s?|\s?inst\S*\s?|\sres\S*\sfound\S*\s?')
            inst_type = pattern.findall(name)[0]
            names.append(names[-1]+inst_type)
        except:
            pass
        
    if ',' in col_name:
        location_full = [name+re.findall(',.*', col_name)[0][1:] for name in names]
        try:
            location_short = [name+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)]
            location_short.extend([name+'-'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+'/'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+' '+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            names.extend(location_short)
        except:
            pass
        names.extend(location_full)
    return (list(set(names))+lst)[:25]

In [506]:
def rm_brk(s):
    s = str(s)
    return re.sub('\\(.+?\\)','',s)

def ret_zero(lst):
    return lst[0]

def add_left_brk(s):
    s = str(s)
    s = s+')'
    return s

In [599]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [597]:
data = pd.read_csv('../data/all_institutions_live.csv', engine='python')

In [604]:
pwd

'/home/kunal_goyal/Desktop/MereExams/MereExamsBot/notebooks'

In [605]:
data.to_sql('/home/kunal_goyal/Desktop/MereExams/MereExamsBot/notebooks/check.sql',engine)

In [610]:
pd.read_sql('check.sql' ,con=engine)

,index,cid,name,address_line1,address_line2,city,Affiliated_to_University,state,pin_code,area,...,no_of_separate_rooms_for_girls,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell
0,0,C100,Manish Institute of Computer Studies,"S. K. Patel Campus, Kamana Char Rasta",Gandhinagar Ambaji State Highway,Visnagar-384315,"Sankalchand Patel University, Visnagar",Gujarat,384315,2.50,...,1.0,False,False,False,8222.0,17.0,True,True,False,False
1,1,C1000,"VIRANI SCI. & SHRI YOGIJI MAHARAJ ARTS, COMMER...","ATMIYA Group of Institutions, Yogidham Gurukul,","Kalawad Road,","Rajkot, Tehsil - Rajkot, Pin - 360005","Saurashtra University, Rajkot",Gujarat,360005,23.50,...,1.0,True,True,True,26352.0,56.0,True,True,True,False
2,2,C10000,"SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI",BIJAPUR ROAD KATATE NAGAR JAMKHANDI,BIJAPUR ROAD KATATE NAGAR JAMKHANDI,JAMKHANDI,"Rani Channamma University, Belagavi",Karnataka,587301,3.00,...,1.0,False,True,False,NaN,NaN,True,False,False,True
3,3,C10002,"VVS'S ARTS/COMM COLLEGE, BIJAPUR",STATION ROAD DARBAR HIGH SCHOOL CAMPUS,STATION ROAD,BIJAPUR-586104,"Rani Channamma University, Belagavi",Karnataka,586104,2.00,...,NaN,False,False,False,5200.0,NaN,False,False,False,False
4,4,C10004,"GOVT. FIRST GRADE COLLEGE, KOKATANOOR",At Post Kokatanoor - 591 230 Tq: Athani,At Post Kokatanoor Tq: Athani,Athani,"Rani Channamma University, Belagavi",Karnataka,591230,5.00,...,1.0,False,False,False,6200.0,4.0,False,False,False,False
5,5,C10005,"S S S S'S COLLEGE OF BUSINESS ADMINISTRATION, ...","Nehru Nagar, Belguam","Nehru Nagar, Belguam",Belguam,"Rani Channamma University, Belagavi",Karnataka,590010,3.00,...,1.0,False,False,False,1600.0,5.0,False,False,False,False
6,6,C10006,"SVM ARTS/SCI/COMM COLLEGE, ILKAL",Near Bus stand,-,ILKAL - 587125,"Rani Channamma University, Belagavi",Karnataka,587125,36.00,...,1.0,False,False,False,40425.0,30.0,False,True,False,True
7,7,C10007,BASAVESHWARA SCIENCE COLLEGE BAGALKOT,Raichur-Belgavi Road,B.V.V.S Campus Bagalkot - 587101,Bagalkot,"Rani Channamma University, Belagavi",Karnataka,587101,162929.92,...,1.0,False,True,True,60662.0,20.0,True,True,True,False
8,8,C10008,AVVSS SHRI PARAPPA SANGAPPA SAJJAN ARTS COLLEG...,PS SAJJAN ARTS COLLEGE BEWOOR TQ:BAGALKOT,PS SAJJAN ARTS COLLEGE BEWOOR TQ:BAGALKOT,BEWOOR,"Rani Channamma University, Belagavi",Karnataka,587115,3.00,...,1.0,False,False,False,NaN,NaN,False,False,False,False
9,9,C10009,SSVVS KUMARI MONICA BASAVARAJ KANNI B.ED COLLE...,"Opp. Al-Ameen Medical College, Athani Road, Bi...",0,Bijapur,"Rani Channamma University, Belagavi",Karnataka,586101,6.00,...,1.0,False,False,True,NaN,NaN,False,False,False,False


In [507]:
data = pd.read_csv('../data/all_institutions_live.csv', engine='python')
data['name_clean'] = data.name

In [508]:
data.name_clean = data.name_clean.apply(rm_brk)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
data.iloc[14376]['name_clean']

'1- D.P. B.S. Degree college Anoopshahr,   Mob.No.9897671410 Ph.No.05734-275450'

In [509]:
data.loc[data.name_clean.str.contains('\('),'name_clean']=data[data.name_clean.str.contains('\(')].name_clean.apply(add_left_brk)
data.name_clean = data.name_clean.apply(rm_brk)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
# data.loc[data.name_clean.str.contains('\('),'name_clean'].values
data.iloc[14376]['name_clean']

'1- D.P. B.S. Degree college Anoopshahr,   Mob.No.9897671410 Ph.No.05734-275450'

In [510]:
data['name_clean'] = data.name_clean.str.split('Ph:|PH[0-9]|Ph[0-9]|Ph. No|Ph\.|Mob\.No\.|PH\.|ph\.|Address|Dist\.|Ward No|PB[0-9]|No [0-9]|PB [0-9]|Distt\.|Distt|H\.No|PIN-|PIN [0-9]|PIN -|Pin-|village|VILLAGE|Write institute Name here|near|Near|NEAR|DIST-|City Off|DIST:|DIST\.|D\.No|Gat No').apply(ret_zero)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
data.iloc[14376]['name_clean']

'1- D.P. B.S. Degree college Anoopshahr'

In [511]:
def rep_sub_none(string):
    string = str(string)
    string = re.sub('\xa0|\t',' ',string)
    return re.sub('[0-9]{3} [0-9]{3}-|[0-9]{6}-|Dist|"|\)|�|','',string)

def rep_sub_quote(string):
    string = str(string)
    string = re.sub(',\.|PINCODE|PB |NH-[0-9]{1,4},|NH-[0-9]{1,4}|Nh-|NH[0-9]{1,4}|NH [0-9]|^[A-Z][0-9]{1,4} |^[A-Z]-[0-9]{1,4}|^[0-9]{1,4} |^[A-Z][0-9]{1,4} |^\?\?|^\?|\+3 Degree|\+3|\+ 3|^\+','',string)
    return re.sub(';|:|\?\?\?\?|\?\?\?|\?\?|\?|\x92','\'',string)

def rep_sub_fs(string):
    string = str(string)
    return re.sub('\.\.|>','.',string)

def rep_sub_ch(string):
    string = str(string)
    string =  re.sub('---|--','-',string)
    string = re.sub('  ,  |  , | ,  | , ',', ',string)
    return re.sub(',,,|,,|<',',',string)

def rep_digit(string):
    string = str(string)
    return re.sub('[0-9]{1,4}$|[0-9]{1,4}.$','',string)

In [512]:
data.name_clean = data.name_clean.apply(rep_sub_none)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
data.iloc[14376].name_clean

'1- D.P. B.S. Degree college Anoopshahr'

In [513]:
data['name_clean'] = data.name_clean.str.split('-[0-9]{3} [0-9]{3}|-[0-9]{6}|P\.O\.|P\.O|VPO|PB\.NO|- [0-9]{3} [0-9]{3}|- [0-9]{6}|[0-9]{3} [0-9]{3}|[0-9]{6}|Ta:|Di:|TAL:|OPP:|Vill|near|Post Box|P\.B\. No|Tq:|Dt:|Tq.:|Opp:|Opp\.|opp\.|Tal:|Di stt:|TQ:|At:|CITY:|Plot|PLOT|Opposite|Opp\.|OPP\.').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data['name_clean'] = data.name_clean.str.split('P\.B\.No\.|P B No\.|No\.|PO |Post \?|Teh.|At post|Sector|SECTOR|sector|plot|Tal\.|Tehsil|DISTT|VILL-|VIll|VILL\.|NO\.|VILL |POST\.|TQ\.|OPP |OPP-|TAL-|TAL\.|AT\.|#|S NO').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
data.iloc[14376].name_clean

'1- D.P. B.S. Degree college Anoopshahr'

In [514]:
data.name_clean = data.name_clean.apply(rep_sub_quote)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data.name_clean = data.name_clean.apply(rep_sub_fs)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data.name_clean = data.name_clean.apply(rep_sub_ch)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')
data.iloc[14376].name_clean

'1- D.P. B.S. Degree college Anoopshahr'

In [515]:
data.name_clean = data.name_clean.apply(rep_digit)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [516]:
data['name_clean'] = data.name_clean.str.split(' [A-Z]-[0-9]{1,4}|[0-9]{3,4}|N\.H\.|SECT\.|STREET NO-|[0-9]{1,4}th|[0-9]{1,4}rd|[0-9]{1,3} KM|At/,| [0-9]{1,3},|[0-9]{1,4}/|[0-9]{1,4}-').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.lstrip()
data.name_clean = data.name_clean.str.lstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('-')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('?')
data.name_clean = data.name_clean.str.rstrip('&')

In [517]:
# data[['cid','name','name_clean']].to_csv('college_name_cleaned.csv',header=['cid','initial','cleaned'])

In [518]:
data[data.name_clean.str.contains('')].name_clean.values
# data.name_clean = data.name_clean.str.replace(',$','.')
# data.name_clean.

array(['Manish Institute of Computer Studies',
       'VIRANI SCI. & SHRI YOGIJI MAHARAJ ARTS, COMMERCE & SCIENCE - RAJKOT',
       'SIDDARTHA COLLEGE OF EDUCATION,  JAMKHANDI', ...,
       'Bharat Ratana Dr. B. R. Ambedkar University',
       'Delhi Technological University',
       'Guru Gobind Singh Indraprastha University'], dtype=object)

In [519]:
# data[data.name_clean.duplicated()].name_clean.shape
data[data.name.duplicated()].name.shape

(214,)

In [520]:
data.loc[10179,'name_clean'] = data.loc[10179,'name']
data.loc[12091,'name_clean'] = data.loc[12091,'name']
data.loc[14376,'name_clean'] = 'D.P. B.S. Degree college Anoopshahr'
data.loc[31290,'name_clean'] = data.loc[31290,'name']
data.loc[36941,'name_clean'] = data.loc[36941,'name']
data.loc[41308,'name_clean'] = 'JCT POLYTECHNIC COLLEGE'
data.loc[41472,'name_clean'] = data.loc[41472,'name']


In [521]:
data[data['name_clean']=='']

,cid,name,address_line1,address_line2,city,Affiliated_to_University,state,pin_code,area,website,...,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell,name_clean


<h1>Here goes the city cleaning :-) </h1>

<h2>Remember to remove below read data</h2>

In [522]:
# data = pd.read_csv('../data/all_institutions_live.csv', engine='python')
# data['name_clean'] = data.name

In [523]:
data['city_clean'] = data.city
data.loc[8854,'city_clean']

'Bangalore-10.'

In [524]:
data.city_clean = data.city_clean.str.lower()

In [525]:
sum(data.city_clean.isnull())

1816

In [526]:
data.city_clean.fillna('NULL',inplace=True)
data.loc[data.city_clean.str.contains('^[0-9]'),'city_clean']='NULL'

In [527]:
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip(':')
data.city_clean = data.city_clean.str.rstrip()

In [528]:
data['city_clean'] = data.city_clean.str.split('pin| dist|,dist').apply(ret_zero)
data['city_clean'] = data.city_clean.str.split('[0-9]{3} [0-9]{3}|[0-9]{6}|[0-9]{1,5}$').apply(ret_zero)

In [529]:
def make_none_first(string):
    string = str(string)
    return re.sub('po & mandal:|tq & dt|kustagi road,|tk & dt|first lane, |at post -peeth ,|at post tha|at & po\.|at & post|at post','',string)

data.city_clean = data.city_clean.apply(make_none_first)

data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip('-')
data.city_clean = data.city_clean.str.lstrip(':')
data.city_clean = data.city_clean.str.lstrip('.')
data.city_clean = data.city_clean.str.lstrip(',')
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip(':')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip()

In [530]:
data.loc[14655,'city_clean']='chandigarh'
data.loc[1960,'city_clean']='etawah'
data.loc[11580,'city_clean']='vijayawada'
data.loc[35751,'city_clean']='surat'
data.loc[35850,'city_clean']='patan'
data.loc[508,'city_clean']='kurukshetra'
data.loc[21425,'city_clean']='banglore'
data.loc[21756,'city_clean']='sangaria'
data.loc[43721,'city_clean']='perinthalmanna'
data.loc[21587,'city_clean']='murudeshwar'
data.loc[38468,'city_clean']='salt lake city'
data.loc[10529,'city_clean']='nellimarla'
data.loc[28797,'city_clean']='atraulihar'
data.loc[42918,'city_clean']='bareilly'
data.loc[28687,'city_clean']='jambusar'
data.loc[27548,'city_clean']=data.loc[12756,'city_clean']='linch'
data.loc[35077,'city_clean']='virpur'
data.loc[36253,'city_clean']='naigaon'
data.loc[44812,'city_clean']='kanakapura'
data.loc[39102,'city_clean']='neyveli township'
data.loc[38081,'city_clean']='katkol'
data.loc[34620,'city_clean']='valod'


In [531]:
data['city_clean'] = data.city_clean.str.split(' village|opp. | mandal| tehsil| via| near| taluka| taluk| tal | tal:| tal-| tal\.| tah| ta | ta:| ta-| ta\.| tq|\.tq| east| south| west| north| po ').apply(ret_zero)
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip(':')
data.city_clean = data.city_clean.str.rstrip()
# ^ta-|^ta:|^tal:-|^tal-|^tal\.|^ta\.:-|^ta\.|^ta\.:|

In [532]:
def make_none(string):
    string = str(string)
    return re.sub('district -|district-|district:|dictrict|district|distt-|distt\. -|distt \.|distt\.|distt -|distt:|distt :|distt|dist-|dist -|dist\.|dist \.|dist:|dist :|dist|[0-9]|junction|village,|village\.|village-|village|villl|vill&mdl|vill-|vill:|vill\.|villege|vill |mandal:|mandal |tehsil-|tehsil -|tehsil|teh |teh\.|teh-|teh:|vpo|via |via\.|via-|via|near by|near|taluka:|taluka-|taluka\.|taluka |taluka|\.taluk|-taluk|taluk|tal |tal:|tal-|tal$|talik$|talk$|tal\.|tah |tah-|^ta|tq|dt$|tk$|^town|town$|suburban|urban|rural|ps |ps-|p\.s |p\.s\.|^at |post$|post-|post |post:| up$| mp$|north|east|west|^po |^po:|^th :|city :| :|u.p$|v & p\.o\.-|p\.o','',string)

In [533]:
data.city_clean = data.city_clean.apply(rm_brk)
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip(':')
data.city_clean = data.city_clean.str.rstrip()

In [534]:
data.city_clean = data.city_clean.apply(make_none)
data.city_clean = data.city_clean.str.lstrip()
data.city_clean = data.city_clean.str.lstrip('-')
data.city_clean = data.city_clean.str.lstrip(':')
data.city_clean = data.city_clean.str.lstrip('.')
data.city_clean = data.city_clean.str.lstrip(':')
data.city_clean = data.city_clean.str.lstrip('.')
data.city_clean = data.city_clean.str.lstrip(':')
data.city_clean = data.city_clean.str.lstrip(',')
data.city_clean = data.city_clean.str.lstrip()

In [535]:
data.city_clean = data.city_clean.str.rstrip()
data.city_clean = data.city_clean.str.rstrip(',')
data.city_clean = data.city_clean.str.rstrip('.')
data.city_clean = data.city_clean.str.rstrip('-')
data.city_clean = data.city_clean.str.rstrip(':')
data.city_clean = data.city_clean.str.rstrip()

In [536]:
data[data.city_clean.str.contains('\(')].city_clean

1799                        anand (
8051     gunthapally, hayathnagar(m
14618                  fazilka (now
36008                      vansda (
38167             bhosag ( astamode
Name: city_clean, dtype: object

In [537]:
data.loc[data.city_clean.str.contains('\('),'city_clean']=data[data.city_clean.str.contains('\(')].city_clean.apply(add_left_brk)
data.city_clean = data.city_clean.apply(rm_brk)
data.city_clean = data.city_clean.str.rstrip('\)')
data.city_clean = data.city_clean.str.rstrip('\(')

In [538]:
data.loc[data.city_clean=='','city_clean'] = 'NULL'

In [539]:
data.loc[data.city_clean.str.len()<3,'city_clean'] = 'NULL'

In [540]:
data.loc[data.city_clean=='m.p','city_clean']='NULL'
data.loc[data.city_clean=='nil','city_clean']='NULL'

<h1>Google merging</h1>

In [541]:
google = pd.read_csv('google_city.csv')

In [542]:
google = google.drop('Unnamed: 0',axis=1)

In [543]:
sum(google.duplicated())

3938

In [544]:
google = google.drop_duplicates()

In [545]:
google.loc[np.logical_and(google.locality.isnull(),google.administrative_area_level_2.notnull()),'locality'] = google[np.logical_and(google.locality.isnull(),google.administrative_area_level_2.notnull())]['administrative_area_level_2']

In [546]:
google[google.locality.isnull()]

,place_id,sublocality_level_1,locality,administrative_area_level_2,administrative_area_level_1,country,postal_code,sublocality_level_2,route,point_of_interest,...,subpremise,sublocality_level_3,postal_town,natural_feature,floor,administrative_area_level_3,postal_code_suffix,post_box,political,administrative_area_level_4


In [547]:
google = google[['place_id','locality']]
google = google.drop_duplicates()

In [548]:
google.locality = google.locality.apply(rm_brk)

In [549]:
google_all = pd.read_csv('../data/googledata.csv',header=None)
google_all.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,C100,ChIJM7k39sBPXDkRiU6Om4PD_e8,"SPCE Campus, Visnagar, Gujarat 384315, India",02765 225 505,+91 2765 225 505,Manish Institute of Computer Studies,5.0,2,http://mcavisnagar.org/,https://maps.google.com/?cid=17293193114190499465,23.683418,72.547188,2018-04-03 08:02:41,2018-04-03 16:12:34
1,2,C1000,ChIJqWdamTTKWTkRia51FA2_j4g,"Yogidham Gurukul, Kalawad Road, Maruti Nagar, ...",0281 256 2681,+91 281 256 2681,Shree M. & N. Virani Science College,4.5,5,http://vsc.edu.in/,https://maps.google.com/?cid=9840293773726559881,22.281674,70.766811,2018-04-03 08:02:41,2018-04-03 16:12:41
2,3,C10000,ChIJq6qqWis3xzsRtQ9CQoxdMVo,"Jamkhandi, Karnataka 587301, India",083532 20183,+91 83532 20183,"BLDE Association's Commerce, BHS Arts & TGP Sc...",4.2,5,http://bldeajkd.org/,https://maps.google.com/?cid=6499078594260766645,16.507270,75.277040,2018-04-03 08:02:41,2018-04-03 16:12:52
3,4,C10006,ChIJD_9u3pUMuDsRfsd49c38v9k,"Ilkal, Karnataka 587125, India",099007 66450,+91 99007 66450,"S.V.M Arts, Science & Commerce College Ilkal",4.1,5,http://www.svmdegreecollege.org/,https://maps.google.com/?cid=15690537588298925950,15.967229,76.110374,2018-04-03 08:02:41,2018-04-03 08:16:13
4,5,C10007,ChIJ1XX3Jod4xzsRNASCCm-WoQc,"Ward No 10, Bagalkot, Karnataka 587101, India",083542 20549,+91 83542 20549,B.V.V.S. Basaveshwar Science College,4.0,5,http://www.bscbgk.org/,https://maps.google.com/?cid=549886033177740340,16.184889,75.707152,2018-04-03 08:02:41,2018-04-03 08:16:17


In [550]:
len(google_all[1].unique())

26724

In [551]:
mask = google_all[1].isin(google_all[google_all[1].duplicated()][1].values)

google_all.loc[mask,'null_count'] = np.sum(google_all.loc[mask].isnull(),axis=1).apply(int)
google_all[mask].sort_values(by=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,null_count
21,22,C10031,ChIJKbFi_UJkvzsRNkvgRWNm3hw,"Savagaon Road, Belgaum, Karnataka 591108, India",0831 243 8123,+91 831 243 8123,Angadi Institute Of Technology And Management,4.0,5,http://www.aitmbgm.ac.in/,https://maps.google.com/?cid=2080212654451870518,15.846783,74.474489,2018-04-03 08:02:41,2018-04-03 08:02:41,0.0
8656,8714,C10031,ChIJYSZpg1phvzsRMAAopXIfaFo,"Sangameshwar Nagar, Sangmeshwar Nagar, Belgaum...",096324 51551,+91 96324 51551,A. R. Angadi,NaN,0,NaN,https://maps.google.com/?cid=6514491438249017392,15.885664,74.508864,2018-04-03 08:20:15,2018-04-03 08:20:15,2.0
27,28,C10044,ChIJk1_UCE5lrzsRTuro1Yu_KNg,"Old Polilce Station Building, Near Jyothi Scho...",0821 256 7906,+91 821 256 7906,Government First Grade College,3.0,5,http://gfgc.kar.nic.in/Kuvempunagara,https://maps.google.com/?cid=15575909918571031118,12.279495,76.623633,2018-04-03 08:02:41,2018-04-03 08:02:41,0.0
8677,8747,C10044,ChIJVVVVmWrNuDsRwFE_qU0wgsA,"Jnana Dhara Campus, Kumareshwar Nagar, Near Ne...",0836 274 0886,+91 836 274 0886,Government First Grade College,4.0,5,http://gfgcdwd.in/contact-us/,https://maps.google.com/?cid=13871702912364663232,15.471074,74.992396,2018-04-03 08:22:09,2018-04-03 08:22:09,0.0
29,30,C10048,ChIJr4k9LXprvzsRfAA_eFSIO8I,"Khanapur, Belgaum, Karnataka 591302, India",083362 22461,+91 83362 22461,"Maratha Mandal's Arts and Commerce College, Kh...",2.0,5,NaN,https://maps.google.com/?cid=13995930163359973500,15.639854,74.503086,2018-04-03 08:02:41,2018-04-03 08:02:41,1.0
8684,8758,C10048,ChIJVwRHJK9mvzsRbbAZUtxtRis,"College Road, Lingaraj Campus, Kallehol, Camp,...",0831 240 2351,+91 831 240 2351,K.L.E. Society’s College of Business Administr...,4.2,5,http://www.klecbalc.org/,https://maps.google.com/?cid=3118300585038295149,15.865130,74.506355,2018-04-03 08:22:51,2018-04-03 08:22:51,0.0
73,74,C10206,ChIJ58E-hMpsxzsRvUw8E4EGopI,"Jamakhandi, Bagalkot, Karnataka 586103, India",083532 20183,+91 83532 20183,Bldea's Bba College,NaN,0,NaN,https://maps.google.com/?cid=10566014827207871677,16.397578,75.458250,2018-04-03 08:02:41,2018-04-03 08:02:41,2.0
8759,8873,C10206,ChIJq6qqWis3xzsRtQ9CQoxdMVo,"Jamkhandi, Karnataka 587301, India",083532 20183,+91 83532 20183,"BLDE Association's Commerce, BHS Arts & TGP Sc...",4.3,5,http://bldeajkd.org/,https://maps.google.com/?cid=6499078594260766645,16.507270,75.277040,2018-04-03 08:30:50,2018-04-03 08:30:50,0.0
87,88,C10225,ChIJLYgq-lHkqzsR-HrOIrsdT7E,"Chinna Seeragapadi, Salem, Tamil Nadu 636308, ...",NaN,NaN,Vinayaka Mission's Homeopathic Medical College...,3.8,5,NaN,https://maps.google.com/?cid=12776463357453433592,11.583591,78.059199,2018-04-03 08:02:41,2018-04-03 08:02:41,3.0
8800,8934,C10225,ChIJjT0xUArlqzsRycetCcDpf_E,"NH 47, Sankagiri Main Road, Ariyanur, Salem, T...",0427 398 7000,+91 427 398 7000,Vinayaka Missions University,3.1,5,http://www.vinayakamission.com/,https://maps.google.com/?cid=17401884496188262345,11.595681,78.084230,2018-04-03 08:33:35,2018-04-03 08:33:35,0.0


In [552]:
filtered = google_all[mask].sort_values(by=1).groupby(1).min()

In [553]:
filtered['cid'] = filtered.index

In [554]:
filtered = filtered[['cid','null_count']]

In [555]:
filtered['null_count']=filtered['null_count'].astype('int')
google_all['null_count'] = google_all['null_count'].fillna(-1)
google_all['null_count']=google_all['null_count'].astype('int')

In [556]:
wanted = []
for i in range(filtered.shape[0]):
    mask1 = google_all[1]==filtered.iloc[i]['cid']
    mask2 = google_all[mask1]['null_count']==filtered.iloc[i]['null_count']
    wanted.append(google_all.iloc[mask2.sort_values(ascending=False).index[0]][0])

wanted
filtered = google_all[google_all[0].isin(wanted)]

In [557]:
google_all = google_all.drop(google_all[mask].index).append(filtered)[[1,2]]

In [558]:
print(google.shape)
print(google_all.shape)

(23160, 2)
(26724, 2)


In [559]:
fnl = pd.merge(google_all,google,how='outer',left_on=2,right_on='place_id').dropna().drop(2,axis=1)

In [560]:
tdata = data[data.city_clean=='NULL'][['cid','city_clean']]
tdata.head()

,cid,city_clean
56,C10065,NULL
258,C10298,NULL
369,C10448,NULL
403,C10506,NULL
507,C10650,NULL


In [561]:
merged = pd.merge(tdata,fnl,left_on='cid',right_on=1)
merged.head()

,cid,city_clean,1,place_id,locality
0,C10506,NULL,C10506,ChIJ6ew8-wy2GjkRUi5692os5bk,Kahanpur
1,C10650,NULL,C10650,ChIJ7XGLluILEjkRZgZkDw2pAmM,Jind
2,C10731,NULL,C10731,ChIJS9wzhENMDjkRE5q-PQ5BjWo,Ambala
3,C10757,NULL,C10757,ChIJE4TgS6CuDTkRhU8-3F3h218,Sonipat
4,C10788,NULL,C10788,ChIJG1Zp7PYxDjkRTdXWApqSCSw,Ismailbad


In [562]:
# data[data['cid'].isin(merged['cid'])]
for i in range(merged.shape[0]):
    data.loc[data['cid']==merged.iloc[i]['cid'],'city_clean']=merged.iloc[i]['locality']

In [563]:
data[data['cid'].isin(merged['cid'])]

,cid,name,address_line1,address_line2,city,Affiliated_to_University,state,pin_code,area,website,...,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell,name_clean,city_clean
403,C10506,"Global College of Engg and Tech. , Anandpur Sahib",Garhsankar Road,Anandpur Sahib,NaN,"I K Gujral Punjab Technical University, Jalandhar",Punjab,140117,10.0000,www.gkfindia.com,...,True,True,9735.0,22.0,True,True,True,True,"Global College of Engg and Tech., Anandpur Sahib",Kahanpur
507,C10650,"Aakash College of Education, Jind",VPO GATAULI TEHSIL JULANA,NaN,NaN,"CHAUDHARY RANBIR SINGH UNIVERSITY, JIND",Haryana,126114,0.0000,www.kashcoe.com,...,False,False,NaN,NaN,False,False,False,False,"Aakash College of Education, Jind",Jind
558,C10731,Mata Shanti Devi Memorial College of Edu,Village Abdullagarh,P.O. Kaserlan Kalan,NaN,"CHAUDHARY RANBIR SINGH UNIVERSITY, JIND",Haryana,133205,0.7500,www.shantidevicollegeofeducation.com,...,False,False,9300.0,20.0,True,True,True,True,Mata Shanti Devi Memorial College of Edu,Ambala
578,C10757,Hindu College of Education,VPO- CHAUDHARYWAS,NaN,NaN,"CHAUDHARY RANBIR SINGH UNIVERSITY, JIND",Haryana,125001,4048.0000,www.hinducoe.com,...,False,False,4635.0,10.0,False,False,False,False,Hindu College of Education,Sonipat
597,C10788,"Manish Papneja Memorial S.D. Girls College, Is...","Manish Papneja Memorial S.D. Girls College, Is...",NaN,NaN,"Kurukshetra University, Kurukshetra",Haryana,136129,5.5000,NaN,...,False,False,4180.0,10.0,False,True,False,False,"Manish Papneja Memorial S.D. Girls College, Is...",Ismailbad
629,C10837,S.R.M. College of Education,7TH MILE STONE ROHTAK ROAD ANUPGARH,NaN,NaN,"CHAUDHARY RANBIR SINGH UNIVERSITY, JIND",Haryana,126102,0.0000,www.srmcollegejind.com,...,False,False,NaN,NaN,False,False,False,False,S.R.M. College of Education,Bir Hisar
672,C10897,"S.C.B. Degree College, Ragdha","AT-RAGDHA, PO-SUREIDIHI, VIA-K.C.PUR","PS-BETNOTI, PIN-757029",NaN,"North Orissa University, Mayurbhanj",Odisha,757029,10.2300,www.scbdegreecollege.org.in,...,False,False,2264.0,15.0,True,False,False,True,"S.C.B. Degree College, Ragdha",Nayatarana
684,C10909,"L.K College, Bangriposi","At-Silpunji,Po-Bangriposi",Ps-Bangriposi,NaN,"North Orissa University, Mayurbhanj",Odisha,757032,10.5000,laxmikantacollegebangriposi.in,...,False,False,5748.0,180.0,True,True,True,True,"L.K College, Bangriposi",Mayurbhanj
687,C10912,"Jashipur College, Jashipur","At- Jashipur, P.O.- K. Jashipur, P.S.- Jashipur",NaN,NaN,"North Orissa University, Mayurbhanj",Odisha,757091,10.0000,www.jashipurcollege.com,...,False,False,6616.0,56.0,False,False,False,False,"Jashipur College, Jashipur",Mayurbhanj
690,C10915,"T.T.Mahavidyalaya, Ghatagaon",At/P.o. Ghatgaon,"Dist. Keonjhar, Odisha Pin-758027",NaN,"North Orissa University, Mayurbhanj",Odisha,1,11.4000,www.ttmahavidyalaya.com,...,False,True,15437.0,640.0,True,True,True,True,"T.T.Mahavidyalaya, Ghatagaon",Ghatgaon


In [564]:
data[data.city_clean=='NULL'].shape

(1070, 52)

<h1> initial data backchodi continuing</h1>

In [565]:
data.city_clean = data.city_clean.str.lower()
len(data.city_clean.str.lower().unique())

7695

In [566]:
# tdata = data[['name_clean','city_clean','cid','state','name']]
# tdata.head()\
data.loc[data['city_clean'].str.contains('dist'),'city_clean'] = 'chikhli'

In [567]:
def add_city(row):
    if(row['city_clean']=='null' or ',' in row['name_clean']): 
        row['name_city_cleaned'] = row['name_clean']
    elif row['city_clean'] not in str(row['name_clean']).lower() or ',' not in row['name_clean']:
        row['name_city_cleaned'] = row['name_clean']+', '+row['city_clean'].capitalize() 
    else:
        row['name_city_cleaned'] = row['name_clean']
    return row

In [568]:
data = data.apply(add_city,axis=1)
# tdata['city_clean'][2] in str(tdata['name_clean'][2]).lower()

In [569]:
# duplicate_cid = tdata[tdata['name_clean'].isin(tdata[tdata['name_clean'].duplicated()]['name_clean'].values)].cid.values

In [570]:
# for i in range(len(duplicate_cid)):
#     tdata.loc[tdata['cid']==duplicate_cid[i],'name_clean'] = tdata.loc[tdata['cid']==duplicate_cid[i],'name'] 

In [571]:
# tdata[tdata['name_clean'].isin(tdata[tdata['name_clean'].duplicated()]['name_clean'].values)].sort_values('name_clean')

In [572]:
len(data.city_clean.unique())

7694

In [573]:
# data[['cid','name','city','name_city_cleaned','city_clean']].to_csv('final_name_city_cleaned.csv')

<h1>Synonyms generation</h1>

In [574]:
names = data[['cid','name_city_cleaned']]
names.name_city_cleaned = names.name_city_cleaned.str.replace('_',' ')
names['tname_city_cleaned'] = names['name_city_cleaned']
names['tname_city_cleaned'] = names['tname_city_cleaned'].apply(syn_gen)
names.head()

/home/kunal_goyal/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/kunal_goyal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/kunal_goyal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

,cid,name_city_cleaned,tname_city_cleaned
0,C100,"Manish Institute of Computer Studies, Visnagar","[MICS V, MICSV, MICS/V, MICS-V, manish institu..."
1,C1000,"VIRANI SCI. & SHRI YOGIJI MAHARAJ ARTS, COMMER...","[virani sci. shri yogiji maharaj arts, VSSYMAC..."
2,C10000,"SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI","[SCE-J, siddartha college of education jamkha..."
3,C10002,"VVS'S ARTS/COMM COLLEGE, BIJAPUR","[VAC, VSACC/B, vvs's arts/comm college bijapu..."
4,C10004,"GOVT. FIRST GRADE COLLEGE, KOKATANOOR","[govt. first grade college, GFGCK, GFGC-K, GFG..."


In [575]:
names

,cid,name_city_cleaned,tname_city_cleaned
0,C100,"Manish Institute of Computer Studies, Visnagar","[MICS V, MICSV, MICS/V, MICS-V, manish institu..."
1,C1000,"VIRANI SCI. & SHRI YOGIJI MAHARAJ ARTS, COMMER...","[virani sci. shri yogiji maharaj arts, VSSYMAC..."
2,C10000,"SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI","[SCE-J, siddartha college of education jamkha..."
3,C10002,"VVS'S ARTS/COMM COLLEGE, BIJAPUR","[VAC, VSACC/B, vvs's arts/comm college bijapu..."
4,C10004,"GOVT. FIRST GRADE COLLEGE, KOKATANOOR","[govt. first grade college, GFGCK, GFGC-K, GFG..."
5,C10005,"S S S S'S COLLEGE OF BUSINESS ADMINISTRATION, ...",[s s s s's college of business administration ...
6,C10006,"SVM ARTS/SCI/COMM COLLEGE, ILKAL","[SAC-I, SAC, svm arts/sci/comm college ilkal..."
7,C10007,"BASAVESHWARA SCIENCE COLLEGE BAGALKOT, Bagalkot","[basaveshwara science college bagalkot, basav..."
8,C10008,AVVSS SHRI PARAPPA SANGAPPA SAJJAN ARTS COLLEG...,"[ASPSSAC/B, ASPSSAC-B, avvss shri parappa sang..."
9,C10009,SSVVS KUMARI MONICA BASAVARAJ KANNI B.ED COLLE...,"[SKMBKBCB, ssvvs kumari monica basavaraj kanni..."


In [576]:
names.tname_city_cleaned.values.tolist()
names[list(range(25))] = pd.DataFrame(names.tname_city_cleaned.values.tolist(), index= names.index)

/home/kunal_goyal/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [577]:
names.drop('tname_city_cleaned',axis=1,inplace=True)

/home/kunal_goyal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [578]:
def comma(s):
    if str(s) != str(None):
        return '"'+s+'"'
    return None
def rmv_unnecess(row):
    row['name'] = str(row['name'])
    row['name'] = re.sub('\)',']',row['name']) 
    row['name'] = re.sub('\(','[',row['name'])
    row['name'] = re.sub('\:','',row['name'])
    row['name'] = re.sub('\;','',row['name'])
    row['name'] = re.sub('\<','',row['name'])
    row['name'] = re.sub('\>','',row['name']) 
    return row

In [579]:
# names = names.apply(rmv_unnecess,axis=1)
names = names.apply(comma)


In [580]:
names = names.drop([24],axis=1)

In [581]:
names[['name_city_cleaned',23]] = names[[23,'name_city_cleaned']]

In [582]:
names.to_csv('final_CollegeName.csv',header=None,index=None)

In [583]:
names.iloc[:23000].to_csv('CollegeName1.csv',header=None,index=None)

In [584]:
names.iloc[23000:].to_csv('CollegeName2.csv',header=None,index=None)

In [585]:
# %history -g -f history

In [586]:
names

,cid,name_city_cleaned,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,"""C100""",NaN,"""MICS V""","""MICSV""","""MICS/V""","""MICS-V""","""manish institute computer studies""","""MICS""","""manish institute of computer studies""","""manish institute computer studies visnagar""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Manish Institute of Computer Studies, Visnagar"""
1,"""C1000""",NaN,"""virani sci. shri yogiji maharaj arts""","""VSSYMAC""","""VSSYMA-C""","""virani sci. shri yogiji maharaj arts commerce...","""VSSYMA""","""VSSYMA commerce & science - rajkot""","""virani sci. & shri yogiji maharaj arts""","""VSSYMA/C""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""VIRANI SCI. & SHRI YOGIJI MAHARAJ ARTS, COMME..."
2,"""C10000""",NaN,"""SCE-J""","""siddartha college of education jamkhandi""","""SCE J""","""SCE""","""SCE jamkhandi""","""SCE/J""","""siddartha college education jamkhandi""","""siddartha college education""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI"""
3,"""C10002""",NaN,"""VAC""","""VSACC/B""","""vvs's arts/comm college bijapur""","""VAC B""","""VACB""","""VAC/B""","""vvs's arts/comm college""","""VAC bijapur""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""VVS'S ARTS/COMM COLLEGE, BIJAPUR"""
4,"""C10004""",NaN,"""govt. first grade college""","""GFGCK""","""GFGC-K""","""GFGC/K""","""govt. first grade college kokatanoor""","""GFGC K""","""GFGC kokatanoor""","""GFGC""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""GOVT. FIRST GRADE COLLEGE, KOKATANOOR"""
5,"""C10005""",NaN,"""s s s s's college of business administration ...","""s s s s's college business administration be...","""SSSSCBAB""","""SSSSSCBA belgaum""","""SSSSCBA/B""","""SSSSSCBA-B""","""SSSSSCBA B""","""SSSSCBA B""",...,"""SSSSSCBAB""","""SSSSCBA belgaum""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""S S S S'S COLLEGE OF BUSINESS ADMINISTRATION,..."
6,"""C10006""",NaN,"""SAC-I""","""SAC""","""svm arts/sci/comm college ilkal""","""SACI""","""SAC I""","""SAC ilkal""","""svm arts/sci/comm college""","""SAC/I""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""SVM ARTS/SCI/COMM COLLEGE, ILKAL"""
7,"""C10007""",NaN,"""basaveshwara science college bagalkot""","""basaveshwara science college bagalkot bagalkot""","""BSCB bagalkot""","""BSCB-B""","""BSCB/B""","""BSCB B""","""basaveshwara science college bagalkot""","""basaveshwara science college bagalkot bagalkot""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""BASAVESHWARA SCIENCE COLLEGE BAGALKOT, Bagal..."
8,"""C10008""",NaN,"""ASPSSAC/B""","""ASPSSAC-B""","""avvss shri parappa sangappa sajjan arts college""","""avvss shri parappa sangappa sajjan arts colle...","""ASPSSAC""","""ASPSSACB""","""ASPSSAC B""","""ASPSSAC bewoor""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""AVVSS SHRI PARAPPA SANGAPPA SAJJAN ARTS COLLE..."
9,"""C10009""",NaN,"""SKMBKBCB""","""ssvvs kumari monica basavaraj kanni b.ed coll...","""ssvvs kumari monica basavaraj kanni b.ed coll...","""SKMBKBC/B""","""SKMBKBC B""","""SKMBKBC bijapur""","""SKMBKBC-B""","""SKMBKBC""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""SSVVS KUMARI MONICA BASAVARAJ KANNI B.ED COLL..."


In [587]:
sum(names[23].str.contains('_'))

0

In [588]:
def chck(string):
    return re.match('[\w ,"&\.\'-/@!`\[\]]+$', string) is None

In [589]:
names[names[23].apply(chck)][23].values

array([], dtype=object)

In [590]:
chck('Manish Institute of Computer Studies, Visnagar')

False

In [596]:
names[names.cid=='"C13656"']

,cid,name_city_cleaned,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
3306,"""C13656""",NaN,"""MB""","""mahavidyalaya bansdih ballia""","""MB ballia""","""mahavidyalaya bansdih""","""MB-B""","""MBB""","""MB/B""","""MB B""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Mahavidyalaya Bansdih, Ballia"""
